In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180


import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os


Autosaving every 180 seconds


/tmp/ipykernel_154562/4197309928.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


############### PREDICT NEXT NEURAL TIME SERIES POINT ################ 

Here we are just trying to get the transformer to learn the dynamics of raw (i.e. PCA denoised) neural time series.

#### Step 1: One brain area

- input shape [time_points] = [40000]    # this is just a single time series from the visualizion notebook
- label: [time_points[1:]]                # here we predict the time series but shifted by 1

This is exactly what transformers are developed to do, so we shouldn't have to do too much work to adapt them. We can also smooth or bin the neural data as it's abit noisy. 

##### Major challenges:

1. Figure out how to feed continous time series into the transformer.

There are some methods already out there

https://huggingface.co/blog/time-series-transformers

https://huggingface.co/docs/transformers/model_doc/time_series_transformer


#### Step 2: Multiple brain areas

- input shape [time_points, n_areas] = [40000, 30]     #  
- label: [time_points[1:], 30]                         # 

##### Major challenges:

1. So we would need to extend the above to work with multiple cortical areas...
